# Notebook for preprocessing Wikipedia (Indonesia) dataset

In [1]:
!rm -rf wiki_phoneme
!rm -rf wikipedia_20231101.id.processed

### Initilizing phonemizer and tokenizer

In [2]:
import yaml

config_path = "Configs/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [3]:
import sys
sys.path.insert(0, '/workspace/src/PL-BERT-ID')
from phonemize import phonemize, EnIndPhonemizer

/workspace/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
global_phonemizer = EnIndPhonemizer(ipa=True, keep_stress=True, sep="")

In [5]:
import os
os.environ['TRUST_REMOTE_CODE'] = 'True'

In [6]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('flax-community/gpt2-small-indonesian')

### Process dataset

Since using load_dataset with the Indonesian Wikipedia (id) resulted in errors (e.g., "Not Found"), we will download and load the dataset manually.

You can download the dataset from this link: https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/20231101.id.

In [7]:
from datasets import load_dataset

# Use a glob pattern to load all Parquet files in the 'wikipedia' folder.
# This pattern will search for all files ending with '.parquet' within the folder.
parquet_folder = "/workspace/src/PL-BERT-ID/wikipedia/*.parquet"

try:
    dataset = load_dataset("parquet", data_files=parquet_folder)
    if isinstance(dataset, dict) or hasattr(dataset, "keys"):
        split_name = "train" if "train" in dataset else list(dataset.keys())[0]
        dataset = dataset[split_name]
    print("Dataset loaded successfully!")
    print(dataset)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Dataset loaded successfully!
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 665622
})


In [8]:
root_directory = "./wiki_phoneme" # set up root directory for multiprocessor processing

In [9]:
# Update the process_shard function with better error handling

import os
import time

num_shards = 100

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return f"Shard {i} already exists"
    
    start_time = time.time()
    print('Processing shard %d ...' % i)
    
    try:
        shard = dataset.shard(num_shards=num_shards, index=i)
        print(f'Shard {i}: {len(shard)} examples to process')
        
        # Process in smaller batches to avoid memory issues
        processed_dataset = shard.map(
            lambda t: phonemize(t['text'], global_phonemizer, tokenizer), 
            remove_columns=['text'],
            batch_size=100,  # Process in smaller batches
            desc=f"Phonemizing shard {i}"
        )
        
        if not os.path.exists(directory):
            os.makedirs(directory)
        processed_dataset.save_to_disk(directory)
        
        elapsed = time.time() - start_time
        print(f'Shard {i} completed in {elapsed:.2f} seconds')
        return f"Shard {i} completed successfully"
        
    except Exception as e:
        print(f'Shard {i} failed: {str(e)}')
        return Exception(f"Shard {i} failed: {str(e)}")

In [10]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

#### Note: You will need to run the following cell multiple times to process all shards because some will fail. Depending on how fast you process each shard, you will need to change the timeout to a longer value to make more shards processed before being killed.


In [11]:
# Update the processing cell with better timeout and worker settings

import os
from pebble import ProcessPool
from concurrent.futures import TimeoutError

# Reduce workers to avoid resource contention and increase timeout
max_workers = 24  # Reduced from 20
timeout_seconds = 2400  # Increased to 30 minutes
failed_shards = []

print(f"Processing {num_shards} shards with {max_workers} workers...")
print(f"Timeout set to {timeout_seconds} seconds ({timeout_seconds//60} minutes)")

with ProcessPool(max_workers=max_workers) as pool:
    future = pool.map(process_shard, range(num_shards), timeout=timeout_seconds)
    
    try:
        for i, result in enumerate(future.result()):
            if isinstance(result, Exception):
                print(f"Shard {i} failed: {result}")
                failed_shards.append(i)
            else:
                if i % 10 == 0:  # Progress indicator
                    print(f"Completed {i+1}/{num_shards} shards")
    except Exception as e:
        print(f"Processing error: {e}")

print(f"Processing completed. Failed shards: {len(failed_shards)}")
if failed_shards:
    print(f"Failed shard indices: {failed_shards}")

Processing 100 shards with 24 workers...
Timeout set to 2400 seconds (40 minutes)
Processing shard 0 ...Processing shard 1 ...Processing shard 2 ...
Processing shard 3 ...Processing shard 4 ...Processing shard 5 ...



Processing shard 6 ...Processing shard 8 ...Processing shard 7 ...
Processing shard 9 ...
Processing shard 10 ...


Processing shard 11 ...Processing shard 12 ...Shard 0: 6657 examples to process
Shard 2: 6657 examples to processProcessing shard 16 ...Shard 3: 6657 examples to processShard 1: 6657 examples to processShard 7: 6657 examples to processProcessing shard 15 ...Shard 5: 6657 examples to process

Shard 4: 6657 examples to processProcessing shard 14 ...
Processing shard 13 ...Processing shard 17 ...Shard 8: 6657 examples to processProcessing shard 18 ...Processing shard 19 ...Processing shard 20 ...
Shard 9: 6657 examples to processProcessing shard 21 ...


Shard 6: 6657 examples to processProcessing shard 22 ...




Processing shard 23 ...


Shard 10: 6657 examp

Phonemizing shard 5:  23% 1562/6657 [39:58<55:21,  1.53 examples/s]s]] 

Processing error: [Errno Task timeout] 2400


Phonemizing shard 17:  13% 876/6657 [39:59<58:36,  1.64 examples/s]  ]

Processing shard 24 ...

Processing shard 25 ...Processing shard 26 ...
Shard 24: 6656 examples to processShard 25: 6656 examples to process

Processing shard 27 ...Shard 26: 6656 examples to process

Shard 27: 6656 examples to processProcessing shard 28 ...

Processing shard 29 ...Shard 28: 6656 examples to process

Processing shard 30 ...Shard 29: 6656 examples to process

Processing shard 31 ...Shard 30: 6656 examples to process

Shard 31: 6656 examples to processProcessing shard 32 ...

Processing shard 33 ...Shard 32: 6656 examples to process

Processing shard 34 ...
Shard 34: 6656 examples to processShard 33: 6656 examples to process

Processing shard 35 ...
Processing shard 36 ...Shard 35: 6656 examples to process

Shard 36: 6656 examples to processProcessing shard 37 ...
Shard 37: 6656 examples to process
Processing shard 38 ...

Processing shard 39 ...Shard 38: 6656 examples to process

Processing shard 40 ...
Processing shard 41 ...Shard 40: 6656 examples to processShard 39: 

...

Processing shard 98 ...
Shard 98: 6656 examples to process


Phonemizing shard 77:  19% 1283/6656 [38:46<5:43:58,  3.84s/ examples]

Processing shard 99 ...
Shard 99: 6656 examples to process


Phonemizing shard 99:  43% 2881/6656 [39:55<40:31,  1.55 examples/s]   

Processing completed. Failed shards: 0


In [12]:
# Add this cell before processing to check existing shards

def check_existing_shards():
    """Check which shards already exist"""
    existing_shards = []
    pending_shards = []
    
    for i in range(num_shards):
        directory = root_directory + "/shard_" + str(i)
        if os.path.exists(directory):
            existing_shards.append(i)
        else:
            pending_shards.append(i)
    
    print(f"Existing shards: {len(existing_shards)}/{num_shards}")
    print(f"Pending shards: {len(pending_shards)}")
    
    return pending_shards

# Check existing shards first
pending_shards = check_existing_shards()

if not pending_shards:
    print("All shards already processed!")
else:
    print(f"Processing {len(pending_shards)} pending shards...")

Existing shards: 17/100
Pending shards: 83
Processing 83 pending shards...


In [ ]:
# Process only pending shards

if pending_shards:
    max_workers = 24  # Conservative number of workers
    timeout_seconds = 3000  # 40 minutes timeout
    failed_shards = []
    
    print(f"Processing {len(pending_shards)} pending shards...")
    
    with ProcessPool(max_workers=max_workers) as pool:
        future = pool.map(process_shard, pending_shards, timeout=timeout_seconds)
        
        try:
            for i, result in enumerate(future.result()):
                if isinstance(result, Exception):
                    print(f"Shard {pending_shards[i]} failed: {result}")
                    failed_shards.append(pending_shards[i])
                else:
                    if i % 5 == 0:  # Progress indicator every 5 shards
                        print(f"Progress: {i+1}/{len(pending_shards)} shards completed")
        except Exception as e:
            print(f"Processing error: {e}")
    
    print(f"Processing completed. Failed shards: {len(failed_shards)}")
else:
    print("No pending shards to process!")

Processing 83 pending shards...
Processing shard 2 ...Processing shard 0 ...Processing shard 1 ...Processing shard 3 ...

Processing shard 4 ...

Processing shard 5 ...Processing shard 6 ...Processing shard 8 ...
Processing shard 7 ...
Processing shard 9 ...Processing shard 10 ...

Processing shard 11 ...


Processing shard 14 ...Shard 0: 6657 examples to processShard 3: 6657 examples to processProcessing shard 13 ...Shard 5: 6657 examples to processProcessing shard 15 ...Shard 1: 6657 examples to processShard 6: 6657 examples to processShard 7: 6657 examples to processShard 2: 6657 examples to process
Processing shard 16 ...
Shard 4: 6657 examples to processShard 8: 6657 examples to process

Processing shard 12 ...

Processing shard 20 ...

Processing shard 17 ...Processing shard 19 ...

Processing shard 22 ...


Shard 10: 6657 examples to process
Shard 9: 6657 examples to processProcessing shard 18 ...Processing shard 21 ...




Shard 14: 6657 examples to processShard 15: 6657 exampl

Phonemizing shard 10:   9% 612/6657 [49:56<5:24:37,  3.22s/ examples]]]

Processing error: [Errno Task timeout] 3000


Phonemizing shard 23:  16% 1084/6656 [49:57<3:18:50,  2.14s/ examples]

Processing shard 24 ...
Processing shard 25 ...Shard 24: 6656 examples to process

Processing shard 26 ...Shard 25: 6656 examples to process

Shard 26: 6656 examples to processProcessing shard 27 ...

Shard 27: 6656 examples to processProcessing shard 28 ...

Processing shard 29 ...Shard 28: 6656 examples to process

Processing shard 30 ...
Shard 29: 6656 examples to process
Processing shard 31 ...Shard 30: 6656 examples to process

Processing shard 47 ...Shard 31: 6656 examples to process



...

Phonemizing shard 55:  12% 803/6656 [49:56<8:16:28,  5.09s/ examples] 



Shard 80: 6656 examples to process


Phonemizing shard 61:  13% 840/6656 [49:57<5:56:45,  3.68s/ examples]]

Processing shard 81 ...Processing shard 82 ...

Shard 82: 6656 examples to processProcessing shard 83 ...Processing shard 84 ...
Shard 81: 6656 examples to process


Processing shard 86 ...Shard 84: 6656 examples to processShard 83: 6656 examples to processProcessing shard 85 ...


Processing shard 87 ...
Shard 86: 6656 examples to process

Processing shard 88 ...
Shard 85: 6656 examples to process
Shard 87: 6656 examples to process
Shard 88: 6656 examples to process


Phonemizing shard 78:   0% 0/6656 [00:00<?, ? examples/s]

### Collect all shards to form the processed dataset

In [ ]:
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        print("%s loaded" % o)
    except:
        continue

In [ ]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk(config['data_folder'])
print('Dataset saved to %s' % config['data_folder'])

In [ ]:
# check the dataset size
dataset

### Remove unneccessary tokens from the pre-trained tokenizer
The pre-trained tokenizer contains a lot of tokens that are not used in our dataset, so we need to remove these tokens. We also want to predict the word in lower cases because cases do not matter that much for TTS. Pruning the tokenizer is much faster than training a new tokenizer from scratch. 

In [ ]:
from simple_loader import FilePathDataset, build_dataloader

file_data = FilePathDataset(dataset)
loader = build_dataloader(file_data, num_workers=32, batch_size=128)

In [ ]:
special_token = config['dataset_params']['word_separator']

In [ ]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    unique_index.extend(batch)
    unique_index = list(set(unique_index))

In [ ]:
# get each token's lower case

lower_tokens = []
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    if word.lower() != word:
        t = tokenizer.encode([word.lower()])[0]
        lower_tokens.append(t)
    else:
        lower_tokens.append(t)

In [ ]:
lower_tokens = (list(set(lower_tokens)))

In [ ]:
# redo the mapping for lower number of tokens

token_maps = {}
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    word = word.lower()
    new_t = tokenizer.encode([word.lower()])[0]
    token_maps[t] = {'word': word, 'token': lower_tokens.index(new_t)}

In [ ]:
import pickle
with open(config['dataset_params']['token_maps'], 'wb') as handle:
    pickle.dump(token_maps, handle)
print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

### Test the dataset with dataloader


In [ ]:
from dataloader import build_dataloader

train_loader = build_dataloader(dataset, batch_size=32, num_workers=0, dataset_config=config['dataset_params'])

In [ ]:
_, (words, labels, phonemes, input_lengths, masked_indices) = next(enumerate(train_loader))

In [ ]:
# Inspect shard content - lihat isi phoneme dari shard tertentu

from datasets import load_from_disk

def inspect_shard(shard_number, num_samples=5):
    """
    Melihat isi phoneme dari shard tertentu
    
    Args:
        shard_number: nomor shard yang ingin dilihat (0-99)
        num_samples: jumlah contoh yang ingin ditampilkan
    """
    shard_directory = f"{root_directory}/shard_{shard_number}"
    
    if not os.path.exists(shard_directory):
        print(f"Shard {shard_number} tidak ada di {shard_directory}")
        return
    
    try:
        # Load shard
        shard_data = load_from_disk(shard_directory)
        print(f"Shard {shard_number} info:")
        print(f"- Total samples: {len(shard_data)}")
        print(f"- Columns: {shard_data.column_names}")
        print("="*60)
        
        # Tampilkan beberapa contoh
        for i in range(min(num_samples, len(shard_data))):
            sample = shard_data[i]
            print(f"\nSample {i+1}:")
            print(f"Input IDs: {sample['input_ids'][:10]}{'...' if len(sample['input_ids']) > 10 else ''}")
            print(f"Phonemes: {sample['phonemes'][:10]}{'...' if len(sample['phonemes']) > 10 else ''}")
            
            # Decode beberapa token untuk melihat isi sebenarnya
            if len(sample['input_ids']) > 0:
                decoded_words = [tokenizer.decode([token_id]) for token_id in sample['input_ids'][:5]]
                print(f"Decoded words (first 5): {decoded_words}")
            
            print(f"Phonemes (first 5): {sample['phonemes'][:5]}")
            print("-"*40)
            
    except Exception as e:
        print(f"Error loading shard {shard_number}: {e}")

# Contoh penggunaan: lihat isi shard 0
inspect_shard(0, num_samples=3)

In [ ]:
import pandas as pd
from tqdm import tqdm

def create_excel_from_dataset(dataset_sample, filename="wikipedia_processed.xlsx"):
    """
    Membuat file Excel dari sampel dataset yang telah diproses.
    
    Args:
        dataset_sample: Sampel dari dataset yang akan diproses.
        filename: Nama file Excel yang akan disimpan.
    """
    processed_data = []
    
    print(f"Processing {len(dataset_sample)} samples to create Excel file...")
    
    for sample in tqdm(dataset_sample):
        input_ids = sample['input_ids']
        phonemes = sample['phonemes']
        
        # Decode input_ids kembali menjadi teks
        # Setiap ID di-decode secara terpisah dan digabungkan dengan spasi
        decoded_text = ' '.join([tokenizer.decode([token_id]) for token_id in input_ids])
        
        # Gabungkan daftar phonemes menjadi satu string
        phonemes_str = ' '.join(phonemes)
        
        processed_data.append({
            'text': decoded_text,
            'phonemes': phonemes_str,
            'input_ids': str(input_ids)  # Simpan sebagai string agar mudah dibaca di Excel
        })
        
    # Buat DataFrame dari data yang diproses
    df = pd.DataFrame(processed_data)
    
    # Simpan DataFrame ke file Excel
    df.to_excel(filename, index=False)
    print(f"\nDataset berhasil disimpan ke {filename}")

# Ambil sebagian kecil dari dataset untuk dibuatkan Excel (misalnya 1000 baris pertama)
num_samples_for_excel = 1000

# TAMBAHKAN KODE INI UNTUK MENJALANKAN FUNGSI:
if len(dataset) > 0:
    # Ambil sampel dari dataset
    dataset_sample = dataset.select(range(min(num_samples_for_excel, len(dataset))))
    
    # Jalankan fungsi untuk membuat Excel
    create_excel_from_dataset(dataset_sample, "wikipedia_processed.xlsx")
else:
    print("Dataset kosong atau belum dimuat!")